In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import gc
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_numeric_features, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days
from func.ml_utils import save_feature, get_cnt_feature, get_dummie_feature, get_label_feature
from func.parallel_utils import get_parallel_arg_list
from joblib import delayed, Parallel

In [2]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, 'ProductCD']

train_paths = glob('../feature/eda_base/*_train.gz')
test_paths = glob('../feature/eda_base/*_test.gz')
train_paths += glob('../feature/org_use/526*_train.gz')
test_paths  += glob('../feature/org_use/526*_test.gz')
train_paths += glob('../feature/raw_use/ker__uid*_train.gz')
test_paths  += glob('../feature/raw_use/ker__uid*_test.gz')

train_paths = [path for path in train_paths 
               if (path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
               or path.count('V')
               or path.count('C')
               or path.count('D')
               or (path.count('526') and path.count('mean'))
               or path.count('uid_')
               or path.count('uid2_t')
               or path.count('uid3_t')
#                or path.count('uid4_t')
#                or path.count('uid5_t')
               or path.count('card')
               or path.count('addr')
               or path.count('domain')
               or path.count('TransactionAmt')
               or path.count('Product')
               )
               and not path.count('fill')
               and not path.count('bin')
               and not path.count('129')
               and not path.count('130')
              ]
test_paths = [path for path in test_paths 
               if (path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
               or path.count('V')
               or path.count('C')
               or path.count('D')
               or (path.count('526') and path.count('mean'))
               or path.count('uid_')
               or path.count('uid2_t')
               or path.count('uid3_t')
#                or path.count('uid4_t')
#                or path.count('uid5_t')
               or path.count('card')
               or path.count('addr')
               or path.count('domain')
               or path.count('TransactionAmt')
               or path.count('Product')
               )
               and not path.count('fill')
               and not path.count('bin')
               and not path.count('129')
               and not path.count('130')
              ]

df_train = parallel_load_data(train_paths)
df_test = parallel_load_data(test_paths)
data = pd.concat([df_train, df_test], axis=0, ignore_index=True)
train_length = df_train.shape[0]

In [3]:
START_DATE = '2017-12-01'
# START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')

data['datetime'] = data['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x) - datetime.timedelta(seconds = 14400) ))
data['datetime'].fillna(datetime.date(2020, 1, 1), inplace=True)
data['date'] = data['datetime'].map(lambda x: x.date())

list_regist = []
for d, diff in tqdm(data[['date', 'D1']].values):
    if diff < 999999:
        regist = date_add_days(d, -1*diff)
    else:
        regist = date_add_days(d, 0)
    list_regist.append(str(regist))

data['Regist_date'] = list_regist

100%|██████████| 1097231/1097231 [00:04<00:00, 262011.78it/s]


In [4]:
# cols_V = [col for col in data.columns if col.startswith('V') and col.count('mean')] + ['V258']
# cols_C = [col for col in data.columns if col.startswith('C')]
cols_D = [col for col in data.columns if col.startswith('D')]
# cols_D = ['D1', 'D3', 'D6', 'D14', 'D15']
col_pd = 'ProductCD'

In [5]:
#========================================================================
# ProductCDあたりのC
#========================================================================
cols_D = [col for col in data.columns if col.startswith('D') and not col.count('Product')]
cols_pcd = data['ProductCD'].unique()

for col in tqdm(cols_D):
    for pcd in cols_pcd:
        feature_name = f'{col}__ProductCD-{pcd}'
        data[feature_name] = -1
        data.loc[data['ProductCD'].isin([pcd]), feature_name] = data.loc[data['ProductCD'].isin([pcd]), col]

100%|██████████| 15/15 [00:35<00:00,  2.47s/it]


In [9]:
#========================================================================
# FE Aggregation User ID & TimeSeries Date
#========================================================================

# User別に期間を切って集計できる様にする
dir_save = 'valid_use'
prefix = '530'


def parallel_agg(df, base_key, feature):
    if str(type(base_key)).count('list'):
        fname = '-'.join(base_key)
    else:
        fname = base_key

    result = df.groupby(base_key)[feature].agg({
        f'{fname}_agg_{feature}_mean': 'mean',
        f'{fname}_agg_{feature}_std': 'std',
    })
    return result


cols_V = [col for col in data.columns if col.startswith('V') and col.count('mean')] + ['V258']
cols_C = [col for col in data.columns if col.startswith('C')]
    
# cols_feature = cols_C
# cols_feature = cols_V
# cols_feature = cols_D
cols_feature = ['TransactionAmt'] + cols_C + cols_D

cols_ugr = sorted([col for col in data.columns if col.count('D') and col not in COLUMNS_IGNORE
                 and col.count('ProductCD-')
                ])

for col in cols_feature:
    data[col] = data[col].astype('float')

# for base_key in tqdm(cols_C + cols_V + cols_D):
for i, base_key in tqdm(enumerate(cols_ugr)):
    
    if str(type(base_key)).count('list'):
        pass
    else:
        base_key = [base_key]
    
    base_train = data[base_key].iloc[:len(df_train)]
    base_test  = data[base_key].iloc[len(df_train):]

    list_p = Parallel(60)([delayed(parallel_agg)(data[base_key + [feature]], base_key, feature) for feature in cols_feature])

    df_agg = pd.concat(list_p, axis=1)
    df_agg.reset_index(inplace=True)

    base_train_agg = base_train.merge(df_agg, how='left', on=base_key)
    base_test_agg = base_test.merge(df_agg, how='left', on=base_key)

    cols_save = [col for col in base_train_agg.columns if col.count('agg')]

    save_feature(base_train_agg[cols_save], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
    save_feature(base_test_agg[cols_save],  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

0it [00:00, ?it/s]

(590540,) | D10__ProductCD-C_agg_TransactionAmt_mean
(590540,) | D10__ProductCD-C_agg_TransactionAmt_std
(590540,) | D10__ProductCD-C_agg_C1_mean
(590540,) | D10__ProductCD-C_agg_C1_std
(590540,) | D10__ProductCD-C_agg_C10_mean
(590540,) | D10__ProductCD-C_agg_C10_std
(590540,) | D10__ProductCD-C_agg_C11_mean
(590540,) | D10__ProductCD-C_agg_C11_std
(590540,) | D10__ProductCD-C_agg_C12_mean
(590540,) | D10__ProductCD-C_agg_C12_std
(590540,) | D10__ProductCD-C_agg_C13_mean
(590540,) | D10__ProductCD-C_agg_C13_std
(590540,) | D10__ProductCD-C_agg_C14_mean
(590540,) | D10__ProductCD-C_agg_C14_std
(590540,) | D10__ProductCD-C_agg_C2_mean
(590540,) | D10__ProductCD-C_agg_C2_std
(590540,) | D10__ProductCD-C_agg_C3_mean
(590540,) | D10__ProductCD-C_agg_C3_std
(590540,) | D10__ProductCD-C_agg_C4_mean
(590540,) | D10__ProductCD-C_agg_C4_std
(590540,) | D10__ProductCD-C_agg_C5_mean
(590540,) | D10__ProductCD-C_agg_C5_std
(590540,) | D10__ProductCD-C_agg_C6_mean
(590540,) | D10__ProductCD-C_agg_C

1it [02:31, 151.72s/it]

(590540,) | D10__ProductCD-H_agg_TransactionAmt_mean
(590540,) | D10__ProductCD-H_agg_TransactionAmt_std
(590540,) | D10__ProductCD-H_agg_C1_mean
(590540,) | D10__ProductCD-H_agg_C1_std
(590540,) | D10__ProductCD-H_agg_C10_mean
(590540,) | D10__ProductCD-H_agg_C10_std
(590540,) | D10__ProductCD-H_agg_C11_mean
(590540,) | D10__ProductCD-H_agg_C11_std
(590540,) | D10__ProductCD-H_agg_C12_mean
(590540,) | D10__ProductCD-H_agg_C12_std
(590540,) | D10__ProductCD-H_agg_C13_mean
(590540,) | D10__ProductCD-H_agg_C13_std
(590540,) | D10__ProductCD-H_agg_C14_mean
(590540,) | D10__ProductCD-H_agg_C14_std
(590540,) | D10__ProductCD-H_agg_C2_mean
(590540,) | D10__ProductCD-H_agg_C2_std
(590540,) | D10__ProductCD-H_agg_C3_mean
(590540,) | D10__ProductCD-H_agg_C3_std
(590540,) | D10__ProductCD-H_agg_C4_mean
(590540,) | D10__ProductCD-H_agg_C4_std
(590540,) | D10__ProductCD-H_agg_C5_mean
(590540,) | D10__ProductCD-H_agg_C5_std
(590540,) | D10__ProductCD-H_agg_C6_mean
(590540,) | D10__ProductCD-H_agg_C

2it [04:30, 141.90s/it]

...

73it [2:08:09, 119.61s/it]

(590540,) | D9__ProductCD-S_agg_TransactionAmt_mean
(590540,) | D9__ProductCD-S_agg_TransactionAmt_std
(590540,) | D9__ProductCD-S_agg_C1_mean
(590540,) | D9__ProductCD-S_agg_C1_std
(590540,) | D9__ProductCD-S_agg_C10_mean
(590540,) | D9__ProductCD-S_agg_C10_std
(590540,) | D9__ProductCD-S_agg_C11_mean
(590540,) | D9__ProductCD-S_agg_C11_std
(590540,) | D9__ProductCD-S_agg_C12_mean
(590540,) | D9__ProductCD-S_agg_C12_std
(590540,) | D9__ProductCD-S_agg_C13_mean
(590540,) | D9__ProductCD-S_agg_C13_std
(590540,) | D9__ProductCD-S_agg_C14_mean
(590540,) | D9__ProductCD-S_agg_C14_std
(590540,) | D9__ProductCD-S_agg_C2_mean
(590540,) | D9__ProductCD-S_agg_C2_std
(590540,) | D9__ProductCD-S_agg_C3_mean
(590540,) | D9__ProductCD-S_agg_C3_std
(590540,) | D9__ProductCD-S_agg_C4_mean
(590540,) | D9__ProductCD-S_agg_C4_std
(590540,) | D9__ProductCD-S_agg_C5_mean
(590540,) | D9__ProductCD-S_agg_C5_std
(590540,) | D9__ProductCD-S_agg_C6_mean
(590540,) | D9__ProductCD-S_agg_C6_std
(590540,) | D9__Pr

74it [2:09:24, 106.21s/it]

(590540,) | D9__ProductCD-W_agg_TransactionAmt_mean
(590540,) | D9__ProductCD-W_agg_TransactionAmt_std
(590540,) | D9__ProductCD-W_agg_C1_mean
(590540,) | D9__ProductCD-W_agg_C1_std
(590540,) | D9__ProductCD-W_agg_C10_mean
(590540,) | D9__ProductCD-W_agg_C10_std
(590540,) | D9__ProductCD-W_agg_C11_mean
(590540,) | D9__ProductCD-W_agg_C11_std
(590540,) | D9__ProductCD-W_agg_C12_mean
(590540,) | D9__ProductCD-W_agg_C12_std
(590540,) | D9__ProductCD-W_agg_C13_mean
(590540,) | D9__ProductCD-W_agg_C13_std
(590540,) | D9__ProductCD-W_agg_C14_mean
(590540,) | D9__ProductCD-W_agg_C14_std
(590540,) | D9__ProductCD-W_agg_C2_mean
(590540,) | D9__ProductCD-W_agg_C2_std
(590540,) | D9__ProductCD-W_agg_C3_mean
(590540,) | D9__ProductCD-W_agg_C3_std
(590540,) | D9__ProductCD-W_agg_C4_mean
(590540,) | D9__ProductCD-W_agg_C4_std
(590540,) | D9__ProductCD-W_agg_C5_mean
(590540,) | D9__ProductCD-W_agg_C5_std
(590540,) | D9__ProductCD-W_agg_C6_mean
(590540,) | D9__ProductCD-W_agg_C6_std
(590540,) | D9__Pr

75it [2:10:47, 99.13s/it] 


In [10]:
#========================================================================
# FE Aggregation User ID & TimeSeries Date
#========================================================================

# User別に期間を切って集計できる様にする
dir_save = 'valid_use'
prefix = '530'


def parallel_agg(df, base_key, feature):
    if str(type(base_key)).count('list'):
        fname = '-'.join(base_key)
    else:
        fname = base_key

    result = df.groupby(base_key)[feature].agg({
        f'{fname}_agg_{feature}_mean': 'mean',
        f'{fname}_agg_{feature}_std': 'std',
    })
    return result


cols_V = [col for col in data.columns if col.startswith('V') and col.count('mean')] + ['V258']
cols_C = [col for col in data.columns if col.startswith('C')]
    
# cols_feature = cols_C
# cols_feature = cols_V
# cols_feature = cols_D
cols_feature = ['TransactionAmt'] + cols_C + cols_D

cols_ugr = sorted([col for col in data.columns if col.count('D') and col not in COLUMNS_IGNORE
                 and col.count('ProductCD-')
                ])

for col in cols_feature:
    data[col] = data[col].astype('float')

# for base_key in tqdm(cols_C + cols_V + cols_D):
for i, base_key in tqdm(enumerate(cols_ugr)):
    
    base_key = ['Regist_date', base_key]
    
    base_train = data[base_key].iloc[:len(df_train)]
    base_test  = data[base_key].iloc[len(df_train):]

    list_p = Parallel(60)([delayed(parallel_agg)(data[base_key + [feature]], base_key, feature) for feature in cols_feature])

    df_agg = pd.concat(list_p, axis=1)
    df_agg.reset_index(inplace=True)

    base_train_agg = base_train.merge(df_agg, how='left', on=base_key)
    base_test_agg = base_test.merge(df_agg, how='left', on=base_key)

    cols_save = [col for col in base_train_agg.columns if col.count('agg')]

    save_feature(base_train_agg[cols_save], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
    save_feature(base_test_agg[cols_save],  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

0it [00:00, ?it/s]

(590540,) | Regist_date-D10__ProductCD-C_agg_TransactionAmt_mean
(590540,) | Regist_date-D10__ProductCD-C_agg_TransactionAmt_std
(590540,) | Regist_date-D10__ProductCD-C_agg_C1_mean
(590540,) | Regist_date-D10__ProductCD-C_agg_C1_std
(590540,) | Regist_date-D10__ProductCD-C_agg_C10_mean
(590540,) | Regist_date-D10__ProductCD-C_agg_C10_std
(590540,) | Regist_date-D10__ProductCD-C_agg_C11_mean
(590540,) | Regist_date-D10__ProductCD-C_agg_C11_std
(590540,) | Regist_date-D10__ProductCD-C_agg_C12_mean
(590540,) | Regist_date-D10__ProductCD-C_agg_C12_std
(590540,) | Regist_date-D10__ProductCD-C_agg_C13_mean
(590540,) | Regist_date-D10__ProductCD-C_agg_C13_std
(590540,) | Regist_date-D10__ProductCD-C_agg_C14_mean
(590540,) | Regist_date-D10__ProductCD-C_agg_C14_std
(590540,) | Regist_date-D10__ProductCD-C_agg_C2_mean
(590540,) | Regist_date-D10__ProductCD-C_agg_C2_std
(590540,) | Regist_date-D10__ProductCD-C_agg_C3_mean
(590540,) | Regist_date-D10__ProductCD-C_agg_C3_std
(590540,) | Regist_da

1it [01:39, 99.02s/it]

(590540,) | Regist_date-D10__ProductCD-H_agg_TransactionAmt_mean
(590540,) | Regist_date-D10__ProductCD-H_agg_TransactionAmt_std
(590540,) | Regist_date-D10__ProductCD-H_agg_C1_mean
(590540,) | Regist_date-D10__ProductCD-H_agg_C1_std
(590540,) | Regist_date-D10__ProductCD-H_agg_C10_mean
(590540,) | Regist_date-D10__ProductCD-H_agg_C10_std
(590540,) | Regist_date-D10__ProductCD-H_agg_C11_mean
(590540,) | Regist_date-D10__ProductCD-H_agg_C11_std
(590540,) | Regist_date-D10__ProductCD-H_agg_C12_mean
(590540,) | Regist_date-D10__ProductCD-H_agg_C12_std
(590540,) | Regist_date-D10__ProductCD-H_agg_C13_mean
(590540,) | Regist_date-D10__ProductCD-H_agg_C13_std
(590540,) | Regist_date-D10__ProductCD-H_agg_C14_mean
(590540,) | Regist_date-D10__ProductCD-H_agg_C14_std
(590540,) | Regist_date-D10__ProductCD-H_agg_C2_mean
(590540,) | Regist_date-D10__ProductCD-H_agg_C2_std
(590540,) | Regist_date-D10__ProductCD-H_agg_C3_mean
(590540,) | Regist_date-D10__ProductCD-H_agg_C3_std
(590540,) | Regist_da

2it [03:20, 99.64s/it]

...

35it [1:08:11, 109.88s/it]

(590540,) | Regist_date-D2__ProductCD-C_agg_TransactionAmt_mean
(590540,) | Regist_date-D2__ProductCD-C_agg_TransactionAmt_std
(590540,) | Regist_date-D2__ProductCD-C_agg_C1_mean
(590540,) | Regist_date-D2__ProductCD-C_agg_C1_std
(590540,) | Regist_date-D2__ProductCD-C_agg_C10_mean
(590540,) | Regist_date-D2__ProductCD-C_agg_C10_std
(590540,) | Regist_date-D2__ProductCD-C_agg_C11_mean
(590540,) | Regist_date-D2__ProductCD-C_agg_C11_std
(590540,) | Regist_date-D2__ProductCD-C_agg_C12_mean
(590540,) | Regist_date-D2__ProductCD-C_agg_C12_std
(590540,) | Regist_date-D2__ProductCD-C_agg_C13_mean
(590540,) | Regist_date-D2__ProductCD-C_agg_C13_std
(590540,) | Regist_date-D2__ProductCD-C_agg_C14_mean
(590540,) | Regist_date-D2__ProductCD-C_agg_C14_std
(590540,) | Regist_date-D2__ProductCD-C_agg_C2_mean
(590540,) | Regist_date-D2__ProductCD-C_agg_C2_std
(590540,) | Regist_date-D2__ProductCD-C_agg_C3_mean
(590540,) | Regist_date-D2__ProductCD-C_agg_C3_std
(590540,) | Regist_date-D2__ProductCD-C

36it [1:09:59, 109.32s/it]

(590540,) | Regist_date-D2__ProductCD-H_agg_TransactionAmt_mean
(590540,) | Regist_date-D2__ProductCD-H_agg_TransactionAmt_std
(590540,) | Regist_date-D2__ProductCD-H_agg_C1_mean


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

